<a href="https://colab.research.google.com/github/jahan-ggn/Stackoverflow-Tag-Prediction/blob/master/Stackoverflow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')


In [ ]:
project_id = "stackoverflow-bigquery-dataset"

In [ ]:
# code to scrap 10L records from query
import pandas as pd
import numpy as np
from pandas.io import gbq
from pandas import ExcelWriter
from google.cloud import bigquery

def split_to_list(tags):
  string_tags = tags.split("|")
  string_tags = ",".join(string_tags)
  tags_list = string_tags.split(",")
  return tags_list


bigquery_client = bigquery.Client()
query = "SELECT title, tags FROM bigquery-public-data.stackoverflow.stackoverflow_posts WHERE title != '' LIMIT 1000000"
stackoverflow_posts = gbq.read_gbq(query, "stackoverflow-bigquery-dataset")
for i in range(0, len(stackoverflow_posts.tags)):
  print(i)
  stackoverflow_posts.tags[i] = split_to_list(stackoverflow_posts.tags[i])
writer = ExcelWriter('output.xlsx')
stackoverflow_posts.to_excel(writer, index=False)
writer.save()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import pandas as pd
excel_sheet_in_dataframe = pd.read_excel('/content/drive/My Drive/Stackoverflow/output.xlsx') 
print(excel_sheet_in_dataframe.head())
print(len(excel_sheet_in_dataframe))

In [ ]:
tag_freq_dict = {}
for tags in excel_sheet_in_dataframe.tags:
  tags = tags.strip('][').split(', ') 
  tags = [t.replace("'","") for t in tags] 
  for tag in tags:
    if tag not in tag_freq_dict:
      tag_freq_dict[tag] = 1
    else:
      tag_freq_dict[tag] += 1

tag_freq_dict_sorted = sorted(tag_freq_dict, key=tag_freq_dict.get, reverse=True)
print(len(tag_freq_dict_sorted))
top500 = tag_freq_dict_sorted[:500]
print(len(top500))
print(top500)

17505
500
['javascript', 'java', 'c#', 'php', 'android', 'jquery', 'python', 'html', 'c++', 'ios', 'mysql', 'css', 'sql', 'asp.net', 'objective-c', 'ruby-on-rails', '.net', 'c', 'iphone', 'arrays', 'angularjs', 'ruby', 'sql-server', 'regex', 'json', 'asp.net-mvc', 'xml', 'r', 'ajax', 'node.js', 'linux', 'wpf', 'django', 'database', 'string', 'vb.net', 'xcode', 'windows', 'swift', 'eclipse', 'excel', 'html5', 'wordpress', 'spring', 'multithreading', 'image', 'forms', 'facebook', 'winforms', 'git', 'bash', 'osx', 'algorithm', 'oracle', 'twitter-bootstrap', 'performance', 'entity-framework', 'vba', 'matlab', 'mongodb', 'linq', 'apache', 'postgresql', 'swing', 'css3', 'visual-studio', 'scala', 'list', 'ruby-on-rails-3', 'perl', 'hibernate', 'shell', 'qt', 'sqlite', 'visual-studio-2010', '.htaccess', 'web-services', 'validation', 'function', 'python-2.7', 'asp.net-mvc-3', 'unit-testing', 'wcf', 'rest', 'uitableview', 'codeigniter', 'sql-server-2008', 'api', 'actionscript-3', 'loops', 'class

In [ ]:
final_tag_data = []
for tags in excel_sheet_in_dataframe.tags:
  temp = []
  tags = tags.strip('][').split(', ') 
  tags = [t.replace("'","") for t in tags] 
  for tag in tags:
    if tag in top500:
      temp.append(tag)
  final_tag_data.append(temp)
print(final_tag_data[:20])

[['c++', 'algorithm'], [], ['iphone', 'objective-c'], ['c++', 'visual-studio', 'boost', 'debugging'], ['wcf', 'web-services'], ['r', 'ggplot2'], ['sql', 'sql-server', 'sql-server-2005', 'tsql'], ['java', 'c'], ['php', 'templates', 'parameters'], ['c', 'unicode'], ['sql'], ['svn', 'git', 'version-control'], ['ruby-on-rails'], ['sharepoint', 'architecture'], ['mysql'], ['mysql', 'delphi'], ['sql-server', 'regex', 'sql-server-2005', 'sql-server-2008'], ['exception', 'exception-handling'], ['python'], ['xml', 'xslt', 'loops']]


In [ ]:
from sklearn.preprocessing import MultiLabelBinarizer
tag_encoder = MultiLabelBinarizer()
tags_encoded = tag_encoder.fit_transform(final_tag_data)
tags_encoded.shape

(100000, 500)

In [ ]:
# importing necessary libraries
import nltk
nltk.download('stopwords')
nltk.download('punkt')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
# cleaning dataframe
# 1. Removing Punctuations
# 2. Lowercase Convertion
# 3. Removing Stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 
import re
import sys

title_list = []
stop_words = set(stopwords.words('english')) 
remove_punctuations = RegexpTokenizer(r'\w+')
pattern = '[0-9]'
print(len(excel_sheet_in_dataframe.title))

print(excel_sheet_in_dataframe.title[0])
title_list = []
for i in range(0, len(excel_sheet_in_dataframe.title)):
  tokens = remove_punctuations.tokenize(str(excel_sheet_in_dataframe.title.values[i]))
  words = [w.lower() for w in tokens]
  words = [w for w in words if not w in stop_words]
  words = [re.sub(pattern, '', i) for i in words] 
  title_list.append(words)
print(title_list[:20])

100000
O(N^2) shortest path algorithm
[['n', '', 'shortest', 'path', 'algorithm'], ['setting', 'shortcut', 'keys', 'textmate'], ['implement', 'swipe', 'able', 'image', 'stacks', 'like', 'photo', 'app', 'using', 'iphone', 'sdk'], ['debug', 'code', 'uses', 'boost', 'w', 'losing', 'sanity'], ['closing', 'disposing', 'wcf', 'service'], ['add', 'labels', 'plot'], ['select', 'table', 'vs', 'select', 'cola', 'colb', 'etc', 'table', 'interesting', 'behaviour', 'sql', 'server', ''], ['use', 'hex', 'values', 'instead', 'normal', 'base', '', 'numbers'], ['pass', 'parameters', 'php', 'template', 'rendered', 'include'], ['best', 'unicode', 'library', 'c'], ['show', 'one', 'many', 'relationship', '', 'columns', '', 'unique', 'row', 'id', 'comma', 'separated', 'list'], ['dealing', 'svn', 'keyword', 'expansion', 'git', 'svn'], ['best', 'web', 'server', 'ruby', 'rails', 'application'], ['sharepoint', 'use', 'lists', 'database'], ['suppress', 'row', 'count', 'tables', 'added', 'auto_increment', 'n', 'my

In [ ]:
from gensim.models import Word2Vec
W2V_SIZE = 300
W2V_WINDOW = 7
W2V_EPOCH = 32
W2V_MIN_COUNT = 10

w2v_model = Word2Vec(title_list, size=W2V_SIZE, window=W2V_WINDOW, min_count=W2V_MIN_COUNT, workers=8)
vocabs = w2v_model.wv.vocab.keys()
print(vocabs)
vocab_size = len(vocabs)
print("Vocab size", vocab_size)
# Train Word Embeddings
w2v_model.train(title_list, total_examples = len(title_list), epochs=W2V_EPOCH)
w2v_model.save('/content/drive/My Drive/Stackoverflow/SO_word2vec_embeddings.bin')

dict_keys(['n', '', 'shortest', 'path', 'algorithm', 'setting', 'shortcut', 'keys', 'textmate', 'implement', 'swipe', 'able', 'image', 'like', 'photo', 'app', 'using', 'iphone', 'sdk', 'debug', 'code', 'uses', 'boost', 'w', 'losing', 'closing', 'wcf', 'service', 'add', 'labels', 'plot', 'select', 'table', 'vs', 'etc', 'behaviour', 'sql', 'server', 'use', 'hex', 'values', 'instead', 'normal', 'base', 'numbers', 'pass', 'parameters', 'php', 'template', 'rendered', 'include', 'best', 'unicode', 'library', 'c', 'show', 'one', 'many', 'relationship', 'columns', 'unique', 'row', 'id', 'comma', 'separated', 'list', 'dealing', 'svn', 'keyword', 'expansion', 'git', 'web', 'ruby', 'rails', 'application', 'sharepoint', 'lists', 'database', 'suppress', 'count', 'tables', 'added', 'output', 'development', 'mysql', 'wrapper', 'delphi', 'regular', 'expressions', 'servers', 'cause', 'outlook', 'change', 'com', 'addin', 'unhandled', 'exceptions', 'python', 'time', 'equivalent', 'counter', 'inside', 'xs

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# Load pre-trained embeddings
import gensim
w2v_model = gensim.models.word2vec.Word2Vec.load('/content/drive/My Drive/Stackoverflow/SO_word2vec_embeddings.bin')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [ ]:
# Split into train and test set
import numpy as np
from sklearn.model_selection import train_test_split
print(len(excel_sheet_in_dataframe.title))
X_train, X_test, y_train, y_test = train_test_split(np.array(title_list), tags_encoded, test_size=0.2, random_state=42)
print("TRAIN size:", len(X_train))
print("TEST size:", len(X_test))

100000
TRAIN size: 80000
TEST size: 20000


In [ ]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, Dropout
from keras.utils.np_utils import to_categorical

MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

tokenizer = Tokenizer()
tokenizer.fit_on_texts(title_list)
# saving
import pickle
with open('/content/drive/My Drive/Stackoverflow/tokenizer.pickle', 'wb') as handle:
  pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import pickle
with open('/content/drive/My Drive/Stackoverflow/tokenizer.pickle', 'rb') as handle:
  tokenizer = pickle.load(handle)
word_index = tokenizer.word_index
print(word_index)
vocab_size = len(word_index)
print('Found %s unique tokens.' % len(word_index))

{'': 1, 'using': 2, 'c': 3, 'file': 4, 'error': 5, 'data': 6, 'get': 7, 'android': 8, 'php': 9, 'jquery': 10, 'java': 11, 'value': 12, 'use': 13, 'function': 14, 'string': 15, 'array': 16, 'python': 17, 'javascript': 18, 'sql': 19, 'net': 20, 'multiple': 21, 'list': 22, 'class': 23, 'object': 24, 'working': 25, 'server': 26, 'code': 27, 'text': 28, 'image': 29, 'table': 30, 'app': 31, 'one': 32, 'html': 33, 'mysql': 34, 'page': 35, 'query': 36, 'create': 37, 'add': 38, 'set': 39, 'method': 40, 'application': 41, 'values': 42, 'change': 43, 'user': 44, 'database': 45, 'variable': 46, 'form': 47, 'way': 48, 'two': 49, 'without': 50, 'custom': 51, 'view': 52, 'different': 53, 'asp': 54, 'files': 55, 'js': 56, 'work': 57, 'windows': 58, 'another': 59, 'make': 60, 'type': 61, 'button': 62, 'time': 63, 'google': 64, 'api': 65, 'access': 66, 'find': 67, 'rails': 68, 'select': 69, 'ios': 70, 'xml': 71, 'web': 72, 'getting': 73, 'element': 74, 'column': 75, 'json': 76, 'css': 77, 'mvc': 78, 'ne

In [ ]:
# Convert the data to padded sequences
X_train_padded = tokenizer.texts_to_sequences(X_train)
X_train_padded = pad_sequences(X_train_padded, maxlen=MAX_SEQUENCE_LENGTH)

In [ ]:
# Embedding matrix for the embedding layer
W2V_SIZE = 300
embedding_matrix = np.zeros((vocab_size+1, W2V_SIZE))
for word, i in tokenizer.word_index.items():
  if word in w2v_model.wv:
    embedding_matrix[i] = w2v_model.wv[word]
print(embedding_matrix.shape)

(32093, 300)


In [ ]:
import keras.backend as K

# Custom loss function to handle multilabel classification task
def multitask_loss(y_true, y_pred):
    # Avoid divide by 0
    y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
    # Multi-task loss
    return K.mean(K.sum(- y_true * K.log(y_pred) - (1 - y_true) * K.log(1 - y_pred), axis=1))


In [ ]:
import keras 
import tensorflow as tf
from keras.models import Sequential

model = Sequential()
model.add(Embedding(vocab_size+1, W2V_SIZE, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False))
model.add(LSTM(50, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(units = 10000, kernel_initializer = 'glorot_uniform', activation = 'relu'))  
model.add(Dropout(0.35))
model.add(Dense(units = 1150, kernel_initializer = 'glorot_uniform', activation = 'relu'))
model.add(Dropout(0.25))
model.add(Dense(units = 750, kernel_initializer = 'glorot_uniform', activation = 'relu'))
model.add(Dense(500, activation='sigmoid'))
model.summary()
model.compile(loss=multitask_loss, optimizer="adam", metrics=['accuracy'])
# Train Model
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
callbacks = [ ReduceLROnPlateau(monitor='val_loss', patience=5, cooldown=0),
              EarlyStopping(monitor='val_acc', min_delta=1e-4, patience=5)]
BATCH_SIZE = 1024
# validation_data = validation_data.batch(BATCH_SIZE)
history = model.fit(X_train_padded, tf.cast(y_train, dtype='float64'), batch_size=BATCH_SIZE, epochs=15, validation_data=None, validation_split=0.3, verbose=True, callbacks=callbacks)
# Save model
model.save('/content/drive/My Drive/Stackoverflow/Tag_predictor.h5')

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 50, 300)           9627900   
_________________________________________________________________
lstm_1 (LSTM)                (None, 50)                70200     
_________________________________________________________________
dense_4 (Dense)              (None, 10000)             510000    
_________________________________________________________________
dropout_2 (Dropout)          (None, 10000)             0         
_________________________________________________________________
dense_5 (Dense)              (None, 1150)              11501150  
_________________________________________________________________
dropout_3 (Dropout)          (None, 1150)              0         
_________________________________________________________________
dense_6 (Dense)              (None, 750)              

In [ ]:
import numpy as np
from keras.models import load_model
import keras 
import openpyxl


def convert_tags(tup): 
  str =  ' '.join(tup) 
  return str 


m = keras.models.load_model('/content/drive/My Drive/Stackoverflow/Tag_predictor.h5', custom_objects={'multitask_loss': multitask_loss})
print(len(X_test))

r = 1
inp = []
for i in X_test:
  i = " ".join(i)
  print("Query : ", i)
  inp.append(i)
  listt = tokenizer.texts_to_sequences(inp)
  temp = pad_sequences(listt, maxlen=MAX_SEQUENCE_LENGTH)
  final = m.predict(temp)
  rounded = np.round(final)
  tags = tag_encoder.inverse_transform(rounded)
  result = convert_tags(tags[0])
  print("Tags : ", result)
  inp.clear()
  r = r + 1